### Importing Libraries

In [26]:
import pandas as pd
import numpy as np

## Loading and Analysing Dataframe

In [27]:
df = pd.read_csv('../data/raw/train-sample.csv')

In [28]:
print(df.columns)
df.head(3)

Index(['PostId', 'PostCreationDate', 'OwnerUserId', 'OwnerCreationDate',
       'ReputationAtPostCreation', 'OwnerUndeletedAnswerCountAtPostTime',
       'Title', 'BodyMarkdown', 'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5',
       'PostClosedDate', 'OpenStatus'],
      dtype='object')


,PostId,PostCreationDate,OwnerUserId,OwnerCreationDate,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,PostClosedDate,OpenStatus
0,6046168,05/18/2011 14:14:05,543315,09/17/2010 10:15:06,1,2,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,mongodb,NaN,NaN,NaN,NaN,NaN,open
1,4873911,02/02/2011 11:30:10,465076,10/03/2010 09:30:58,192,24,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,dom,xsd,jaxp,NaN,NaN,NaN,open
2,3311559,07/22/2010 17:21:54,406143,07/22/2010 16:58:20,1,0,Too many lookup tables,What are the adverse effects of having too man...,sql-server,database-design,enums,NaN,NaN,NaN,open


### Features To Create/Encode
From these columns, I consider important:
- Title,
- BodyMarkdown,
- SelectedTags (top N tags, rest as "other"),
- user life at creation (PostCreationDate - OwnerCreationDate)
- OpenStatus

I will create them in the next sections

## Data processing

### Statuses

In [29]:
statuses = df['OpenStatus'].unique()
statuses

array(['open', 'too localized', 'not a real question', 'off topic',
       'not constructive'], dtype=object)

### Most Frequent Tags

In [30]:
tag_column_names = ['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5']
tags_long = list(df[tag_column_names].values.ravel('K'))

tags_unique, frequencies = np.unique(tags_long, return_counts=True)
freq_dict = {tags_unique[i]: frequencies[i] for i in range(len(tags_unique))}

In [31]:
tags_freq_arr = sorted(freq_dict.items(), key=lambda kv: -1 * kv[1])

print('there is {} unique tags, some of them (along with frequencies): {}'.format(len(tags_freq_arr), tags_freq_arr[:10]))

there is 18309 unique tags, some of them (along with frequencies): [('nan', 316799), ('php', 13134), ('c#', 12076), ('java', 11870), ('javascript', 8925), ('android', 8502), ('jquery', 6835), ('c++', 6203), ('iphone', 5125), ('python', 4731)]


As we can see, the most frequent value is 'nan' - we do not want to include it so it will be removed in the following cells.

We will also select N the most frequently occuring tags and classify the rest as `other`

In [32]:
N_tags = 500

tags_freq_arr = tags_freq_arr[1:][:N_tags]

selected_tags = np.empty(shape=N_tags, dtype=tags_unique.dtype)
for i, item in enumerate(tags_freq_arr):
  selected_tags[i] = item[0]

### Selecting Best Features

In [33]:
print(df.columns)
df.head(3)

Index(['PostId', 'PostCreationDate', 'OwnerUserId', 'OwnerCreationDate',
       'ReputationAtPostCreation', 'OwnerUndeletedAnswerCountAtPostTime',
       'Title', 'BodyMarkdown', 'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5',
       'PostClosedDate', 'OpenStatus'],
      dtype='object')


,PostId,PostCreationDate,OwnerUserId,OwnerCreationDate,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,PostClosedDate,OpenStatus
0,6046168,05/18/2011 14:14:05,543315,09/17/2010 10:15:06,1,2,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,mongodb,NaN,NaN,NaN,NaN,NaN,open
1,4873911,02/02/2011 11:30:10,465076,10/03/2010 09:30:58,192,24,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,dom,xsd,jaxp,NaN,NaN,NaN,open
2,3311559,07/22/2010 17:21:54,406143,07/22/2010 16:58:20,1,0,Too many lookup tables,What are the adverse effects of having too man...,sql-server,database-design,enums,NaN,NaN,NaN,open


## Data processing

### Most Frequent Tags

In [34]:
N_tags = 500

tags_freq_arr = tags_freq_arr[1:][:N_tags]

selected_tags = np.empty(shape=N_tags, dtype=tags_unique.dtype)
for i, item in enumerate(tags_freq_arr):
  selected_tags[i] = item[0]

### Days Since Account Creation Till Posting Question

In [35]:
df['OwnerCreationDate'] = pd.to_datetime(df['OwnerCreationDate'], format='mixed')
df['PostCreationDate'] = pd.to_datetime(df['PostCreationDate'], format='mixed')

In [36]:
df['DaysTillPosting'] = (df['PostCreationDate'] - df['OwnerCreationDate']).dt.days

### Processing Tags

In [37]:
def getRecognizedTags(row):
  recognized_tags = []
  
  for tag in row[tag_column_names].values:
    if(np.isin(tag, selected_tags)):
      recognized_tags.append(tag)
  
  return recognized_tags

def getUnrecognizedTags(row):
  unrecognized_tags = 0
  
  for tag in row[tag_column_names].values:
    if(~np.isin(tag, selected_tags)):
      unrecognized_tags += 1
  
  return unrecognized_tags

In [38]:
df["RecognizedTags"] = df[:1000].apply(getRecognizedTags, axis=1)
df["UnrecognizedTags"] = df[:1000].apply(getUnrecognizedTags, axis=1)

# REMOVE [:1000] !!!

### Picking important columns

In [39]:
important_columns = ['ReputationAtPostCreation', 'OwnerUndeletedAnswerCountAtPostTime', 'Title', 'BodyMarkdown', 'DaysTillPosting', 'RecognizedTags', 'UnrecognizedTags', 'OpenStatus']
df = df[important_columns]

In [40]:
df.head(5)

,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,DaysTillPosting,RecognizedTags,UnrecognizedTags,OpenStatus
0,1,2,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,243,[mongodb],4.0,open
1,192,24,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,122,[dom],4.0,open
2,1,0,Too many lookup tables,What are the adverse effects of having too man...,0,"[sql-server, database-design]",3.0,open
3,4,1,What is this PHP code in VB.net,I am looking for the vb.net equivalent of this...,258,[vb.net],4.0,too localized
4,334,14,Spring-Data mongodb querying multiple classes ...,"With Spring-Data, you can use the @Document an...",453,[mongodb],4.0,open
